In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115393


In [4]:
# let's look at the first 1000 characters
print(text[:50])

First Citizen:
Before we proceed any further, hear


In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print(chars)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

print(len(encode("hii there")) == len("hii there"))
print(decode([1]))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there
True
 


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(data[:100]) # the 1000 characters we looked at earier will to the GPT look like this
print(decode(data[:100].tolist()))

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
n = int(0.9*len(data))
print(len(data))
print(n)

1115393
1003853


In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
decode(train_data[:block_size+1].tolist())

'First Cit'

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print('x', x)
print('y', y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{t} when input is {context} the target: {target}")

x tensor([18, 47, 56, 57, 58,  1, 15, 47])
y tensor([47, 56, 57, 58,  1, 15, 47, 58])
0 when input is tensor([18]) the target: 47
1 when input is tensor([18, 47]) the target: 56
2 when input is tensor([18, 47, 56]) the target: 57
3 when input is tensor([18, 47, 56, 57]) the target: 58
4 when input is tensor([18, 47, 56, 57, 58]) the target: 1
5 when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
6 when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
7 when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [13]:
ixx = torch.randint(len(train_data) - 8, (4,))
print(ixx)
print(torch.randint(10, (5,)))

print('----')
print(train_data[i:i+2] for i in ixx)
print([i for i in ixx])
print([train_data[i:i+2] for i in ixx])
print([train_data[i:i+8] for i in ixx])
print('----')

print(torch.stack([train_data[i:i+8] for i in ixx]))
print('----')
print(torch.stack([train_data[i+1:i+8+1] for i in ixx]))

tensor([442646, 162070, 717590, 892364])
tensor([4, 8, 2, 2, 9])
----
<generator object <genexpr> at 0x000002AD1EF827A0>
[tensor(442646), tensor(162070), tensor(717590), tensor(892364)]
[tensor([ 1, 42]), tensor([44, 57]), tensor([52, 42]), tensor([ 1, 52])]
[tensor([ 1, 42, 39, 63,  1, 52, 53, 56]), tensor([44, 57, 39, 51, 43,  1, 46, 39]), tensor([52, 42, 43, 43, 42,  8,  0, 35]), tensor([ 1, 52, 53, 52, 43, 11,  0, 21])]
----
tensor([[ 1, 42, 39, 63,  1, 52, 53, 56],
        [44, 57, 39, 51, 43,  1, 46, 39],
        [52, 42, 43, 43, 42,  8,  0, 35],
        [ 1, 52, 53, 52, 43, 11,  0, 21]])
----
tensor([[42, 39, 63,  1, 52, 53, 56,  1],
        [57, 39, 51, 43,  1, 46, 39, 52],
        [42, 43, 43, 42,  8,  0, 35, 46],
        [52, 53, 52, 43, 11,  0, 21, 44]])


In [14]:
print("actual words")
print("step 1")
print([decode(train_data[i:i+8].tolist()) for i in ixx])
print([decode(train_data[i+1:i+8+1].tolist()) for i in ixx])
print("step 2")
print([[j for j in decode(train_data[i+1:i+8+1].tolist())] for i in ixx])
print("step 3")
print("xb")
for i in ixx:
  print([j for j in decode(train_data[i:i+8].tolist())])
print("yb")
for i in ixx:
  print([j for j in decode(train_data[i+1:i+8+1].tolist())])


print(torch.stack([train_data[i:i+8] for i in ixx]))
print('----')
print(torch.stack([train_data[i+1:i+8+1] for i in ixx]))

actual words
step 1
[' day nor', 'fsame ha', 'ndeed.\nW', ' none;\nI']
['day nor ', 'same han', 'deed.\nWh', 'none;\nIf']
step 2
[['d', 'a', 'y', ' ', 'n', 'o', 'r', ' '], ['s', 'a', 'm', 'e', ' ', 'h', 'a', 'n'], ['d', 'e', 'e', 'd', '.', '\n', 'W', 'h'], ['n', 'o', 'n', 'e', ';', '\n', 'I', 'f']]
step 3
xb
[' ', 'd', 'a', 'y', ' ', 'n', 'o', 'r']
['f', 's', 'a', 'm', 'e', ' ', 'h', 'a']
['n', 'd', 'e', 'e', 'd', '.', '\n', 'W']
[' ', 'n', 'o', 'n', 'e', ';', '\n', 'I']
yb
['d', 'a', 'y', ' ', 'n', 'o', 'r', ' ']
['s', 'a', 'm', 'e', ' ', 'h', 'a', 'n']
['d', 'e', 'e', 'd', '.', '\n', 'W', 'h']
['n', 'o', 'n', 'e', ';', '\n', 'I', 'f']
tensor([[ 1, 42, 39, 63,  1, 52, 53, 56],
        [44, 57, 39, 51, 43,  1, 46, 39],
        [52, 42, 43, 43, 42,  8,  0, 35],
        [ 1, 52, 53, 52, 43, 11,  0, 21]])
----
tensor([[42, 39, 63,  1, 52, 53, 56,  1],
        [57, 39, 51, 43,  1, 46, 39, 52],
        [42, 43, 43, 42,  8,  0, 35, 46],
        [52, 53, 52, 43, 11,  0, 21, 44]])


In [15]:
print("single line")
print("==============")
for i in ixx:
  print([decode(train_data[i:i+130].tolist())])
  break

print("multi line")
print("==============")
for i in ixx:
  print(decode(train_data[i:i+130].tolist()))
  break

print("batch single line")
print("==============")
for i in ixx:
  print([decode(train_data[i:i+130].tolist())])

single line
[' day nor light.\nLords, I protest, my soul is full of woe,\nThat blood should sprinkle me to make me grow:\nCome, mourn with me for t']
multi line
 day nor light.
Lords, I protest, my soul is full of woe,
That blood should sprinkle me to make me grow:
Come, mourn with me for t
batch single line
[' day nor light.\nLords, I protest, my soul is full of woe,\nThat blood should sprinkle me to make me grow:\nCome, mourn with me for t']
['fsame hand that made these wounds!\nLo, in these windows that let forth thy life,\nI pour the helpless balm of my poor eyes.\nCursed ']
["ndeed.\nWhy, lo you now, I have spoke to the purpose twice:\nThe one for ever earn'd a royal husband;\nThe other for some while a fri"]
[" none;\nIf you be one, as you are well express'd\nBy all external warrants, show it now,\nBy putting on the destined livery.\n\nISABELL"]


In [16]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 47
when input is [49, 43, 

In [17]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is = {context.tolist()} (`{decode(context.tolist())}`) the target: {target} ({itos[int(target)]})")
    break

when input is = [53] (`o`) the target: 59 (u)
when input is = [53, 59] (`ou`) the target: 6 (,)
when input is = [53, 59, 6] (`ou,`) the target: 1 ( )
when input is = [53, 59, 6, 1] (`ou, `) the target: 58 (t)
when input is = [53, 59, 6, 1, 58] (`ou, t`) the target: 56 (r)
when input is = [53, 59, 6, 1, 58, 56] (`ou, tr`) the target: 47 (i)
when input is = [53, 59, 6, 1, 58, 56, 47] (`ou, tri`) the target: 40 (b)
when input is = [53, 59, 6, 1, 58, 56, 47, 40] (`ou, trib`) the target: 59 (u)


In [18]:
print(xb) # our input to the transformer

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])


In [25]:
import torch
import torch.nn as nn

num_letters = 2
num_columns = 4
# row_content = [0,1,1,1,0,0]
# num_rows = len(row_content)
em = nn.Embedding(num_letters, num_columns)
em.weight

Parameter containing:
tensor([[ 0.1378, -0.3889,  0.5133,  0.3319],
        [ 0.6300,  0.5815, -0.0282, -0.1744]], requires_grad=True)

In [20]:
import torch
import torch.nn as nn
torch.manual_seed(1337)

num_letters = 4
num_columns = 4
row_content = [
    [0,2],
    [1,3]
]
num_rows = len(row_content)
emm = nn.Embedding(num_letters, num_columns)
# emm.weight = em.weight
print(emm)
print(emm.weight)
# print(emm.weight)
t1 = torch.LongTensor(row_content)
# print(t1)
t1em = emm(t1)
print(t1em.shape)
print(t1em[:,-1,:].shape)
print(t1em[:,-1,:])
# print(t1em)

Embedding(4, 4)
Parameter containing:
tensor([[ 0.1808, -0.0700, -0.3596, -0.9152],
        [ 0.6258,  0.0255,  0.9545,  0.0643],
        [ 0.3612,  1.1679, -1.3499, -0.5102],
        [ 0.2360, -0.2398, -0.9211,  1.5433]], requires_grad=True)
torch.Size([2, 2, 4])
torch.Size([2, 4])
tensor([[ 0.3612,  1.1679, -1.3499, -0.5102],
        [ 0.2360, -0.2398, -0.9211,  1.5433]], grad_fn=<SliceBackward0>)


In [24]:
print(em)
print(torch.LongTensor(row_content))
# print(em(torch.LongTensor([0,1,2,3,4,5,6,7])))
# print(em(torch.LongTensor([0,1,2,2,4,4])))

Embedding(2, 4)
tensor([[0, 2],
        [1, 3]])


In [26]:
em(torch.LongTensor([0, 0]))

tensor([[ 0.1378, -0.3889,  0.5133,  0.3319],
        [ 0.1378, -0.3889,  0.5133,  0.3319]], grad_fn=<EmbeddingBackward0>)

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # print("token_embedding_table ==> ", self.token_embedding_table)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        # print('idx shape --> ',idx.shape)
        # print('idx --> ',idx)
        # print('self.token_embedding_table --> ',self.token_embedding_table)
        logits = self.token_embedding_table(idx) # (B,T,C)
        # print("logits shape --> ",logits.shape)
        # print("logits --> ",logits)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb)
# print(logits.shape)
# print(loss)
# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1)[0].tolist()))


In [28]:
# context = torch.LongTensor([0]) not working
print(encode("hii there"))
print(len(encode("hii there")))
context_custom_text = torch.LongTensor([encode("hii there")])
print(context_custom_text)
context_custom = torch.LongTensor([[0,1,2,3,4]])
context = torch.ones((1, 5), dtype=torch.long)
print(context)
print(decode(m.generate(idx = context_custom_text, max_new_tokens=100)[0].tolist()))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
9
tensor([[46, 47, 47,  1, 58, 46, 43, 56, 43]])
tensor([[1, 1, 1, 1, 1]])
hii therevv?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [31]:
print(m.token_embedding_table)
# batch, times,
# print()

Embedding(65, 65)


In [32]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [33]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4699912071228027


In [34]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henouratucenonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h haybet--s n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scat t tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN the. sE:
I hisgothers je are! ABer wotouciullle'z


# The mathematical trick in self-attention

In [35]:
import torch

In [36]:
a = torch.tril(torch.ones(3,3))
a/= torch.sum(a,1,keepdim=True)
a
torch.randint(0,10,(3,2)).float()

tensor([[2., 2.],
        [0., 9.],
        [3., 0.]])

In [37]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [38]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [39]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [40]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [42]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [43]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [44]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:

- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with **tril**, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides **wei** by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [45]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [46]:
k.var()

tensor(1.0449)

In [47]:
q.var()

tensor(1.0700)

In [48]:
wei.var()

tensor(1.0918)

In [49]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [50]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [51]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [52]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [53]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [54]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

